In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv')

In [12]:
df.shape

(500, 14)

In [9]:
df.isna().sum()

id                  0
name                0
street              0
houseNumber        96
postalCode          0
city                0
state               0
country             0
stationCategory     0
owner               0
countryCode         0
latitude           26
longitude          26
timeZone            0
dtype: int64

In [15]:
missing_values = df[df.isna().any(axis=1)]
print(missing_values)

       id                           name               street houseNumber  \
2    1001                 Burkhardtsdorf        Bahnhofstraße         NaN   
15   1018                      Cainsdorf        Am Hammerwald         NaN   
17    102                  Altenhasungen          Hardtstraße         NaN   
18   1020              Calbe (Saale) Ost          Am Klärwerk         NaN   
21   1023                        Caldern       Zum Wollenberg         NaN   
..    ...                            ...                  ...         ...   
455  1531                      Eisenärzt           Dorfstraße         NaN   
462   154          Annaberg-Buchholz Süd  Schneeberger Straße         NaN   
470   155  Annaberg-Buchholz unt Bahnhof        Bahnhofsplatz         NaN   
478  1559                     Elpersheim   Deutschordenstraße         NaN   
487  1570             Elsterwerda-Biehla       Am Nordbahnhof         NaN   

     postalCode               city              state country stationCatego